In [ ]:
import sys
sys.path.append('../run_analyses/')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# from start_run import *
from support.charting import primaryColors, fullColors, save_fig
from support.load     import loadAllData, methodParams
from support.util     import titles, order, methodTitle, modelTitle

from modelConfig      import models
from util.util        import getLeverNames

from ema_workbench.analysis import parcoords

from IPython.core import display as ICD

In [ ]:
sns.set_style('white')
data = loadAllData()

In [ ]:
robustData = {}
for method in data.keys(): 
    robustData[method] = {}
    for model in data[method]['robusts'].keys(): 
        robustData[method][model] = []
        for idx in range(len(data[method]['robusts'][model])):
            df = data[method]['robusts'][model][idx][order('outcome')]
            df = df.rename(columns={'max_P_percent': 'Fraction\nPollution', 'reliability_percent': 'Fraction\nReliability', 
                                          'utility_percent': 'Fraction\nUtility', 'inertia_percent': 'Fraction\nInertia'})
            robustData[method][model].append(df)

In [ ]:
def robust_parallel_axes(robust, color): 
    sns.set_palette(sns.light_palette(color, n_colors=robust.shape[0]))

    limits = parcoords.get_limits(robust)
    limits.iloc[0, :] = 0
    limits.iloc[1, :] = 1

    axes = parcoords.ParallelAxes(limits, rot=0, fontsize=12)
    
    return axes

def reorderDatalabels(kind='method', axes=None): 
    newOrder = [None, None, None]
    if kind == 'method': 
        for item in axes.datalabels: 
            if item[0] == methodTitle['mordm']: 
                newOrder[0] = item
            elif item[0] == methodTitle['multi']: 
                newOrder[1] = item
            else: 
                newOrder[2] = item
    elif kind == 'model': 
        for item in axes.datalabels: 
            if item[0] == modelTitle['dps'].replace('\n',' '): 
                newOrder[2] = item
            elif item[0] == modelTitle['plannedadaptive'].replace('\n',' '): 
                newOrder[1] = item
            else: 
                newOrder[0] = item
    axes.datalabels = newOrder
    return axes

def parallelRobustSummary(model, robustData, root=''): 
    axes = robust_parallel_axes(robustData['mordm'][model][0], primaryColors[model]['mordm'])
    axes.plot(robustData['mordm'][model][0], linewidth=0.5)
    axes.fig.suptitle(methodTitle['mordm'], fontsize=14, weight='bold', y = 1.03)
    save_fig(axes.fig, 'images/robust','parallel_mordm_' + model)
    plt.show()

    axes = robust_parallel_axes(robustData['multi'][model][0], primaryColors[model]['multi'])
    colors = list(reversed(fullColors[model]['multi']))
    for idx, robust in enumerate(list(reversed(robustData['multi'][model]))): 
        axes.plot(robust, color=colors[idx], linewidth=0.5)
    axes.fig.suptitle(methodTitle['multi'], fontsize=14, weight='bold', y = 1.03)
    save_fig(axes.fig, 'images/robust','parallel_multi_' + model + '_' + root)
    plt.show()

    axes = robust_parallel_axes(robustData['moro'][model][0], primaryColors[model]['moro'])
    axes.plot(robustData['moro'][model][0], linewidth=0.5, color=primaryColors[model]['moro'])
    axes.fig.suptitle(methodTitle['moro'], fontsize=14, weight='bold', y = 1.03)
    save_fig(axes.fig, 'images/robust','parallel_multi_' + model + '_' + root)
    plt.show()

# Separate Pareto Results

### DPS

In [ ]:
parallelRobustSummary('dps',robustData)

### Planned Adaptive DPS

In [ ]:
parallelRobustSummary('plannedadaptive',robustData)

### Intertemporal

In [ ]:
goodInertia = {}
badInertia = {}
for method in robustData.keys(): 
    goodInertia[method] = {'intertemporal':[]}
    badInertia[method] = {'intertemporal':[]}
    for dat in robustData[method]['intertemporal']:
        goodInertia[method]['intertemporal'].append(dat.loc[dat['Fraction\nInertia'] > 0.5])
        badInertia[method]['intertemporal'].append(dat.loc[dat['Fraction\nInertia'] <= 0.5])

In [ ]:
parallelRobustSummary('intertemporal',robustData)

In [ ]:
parallelRobustSummary('intertemporal',goodInertia,'good')

In [ ]:
parallelRobustSummary('intertemporal',badInertia,'bad')

# Comparing Lever Values

In [ ]:
linewidth = 0.5

In [ ]:
key = 'dps'

axes = robust_parallel_axes(robustData['mordm'][key][0], primaryColors[key]['mordm'])

axes.plot(pd.concat(robustData['multi'][key]), color=primaryColors[key]['multi'], label=methodTitle['multi'], linewidth=linewidth)
axes.plot(robustData['mordm'][key][0],color=primaryColors[key]['mordm'], label=methodTitle['mordm'], linewidth=linewidth)   
axes.plot(robustData['moro'][key][0],color=primaryColors[key]['moro'], label=methodTitle['moro'], linewidth=linewidth)

axes = reorderDatalabels(kind='method', axes=axes)
axes.legend()
axes.fig.suptitle(modelTitle[key].replace('\n',' ') + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

In [ ]:
key = 'plannedadaptive'

axes = robust_parallel_axes(robustData['mordm'][key][0], primaryColors[key]['mordm'])

axes.plot(pd.concat(robustData['multi'][key]), color=primaryColors[key]['multi'], label=methodTitle['multi'], linewidth=linewidth)
axes.plot(robustData['mordm'][key][0],color=primaryColors[key]['mordm'], label=methodTitle['mordm'], linewidth=linewidth)   
axes.plot(robustData['moro'][key][0],color=primaryColors[key]['moro'], label=methodTitle['moro'], linewidth=linewidth)

axes = reorderDatalabels(kind='method', axes=axes)
axes.legend()
axes.fig.suptitle(modelTitle[key].replace('\n',' ') + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

In [ ]:
key = 'intertemporal'

axes = robust_parallel_axes(robustData['mordm'][key][0], primaryColors[key]['mordm'])

axes.plot(pd.concat(robustData['multi'][key]), color=primaryColors[key]['multi'], label=methodTitle['multi'], linewidth=linewidth)
axes.plot(robustData['mordm'][key][0],color=primaryColors[key]['mordm'], label=methodTitle['mordm'], linewidth=linewidth)   
axes.plot(robustData['moro'][key][0],color=primaryColors[key]['moro'], label=methodTitle['moro'], linewidth=linewidth)

axes = reorderDatalabels(kind='method', axes=axes)
axes.legend()
axes.fig.suptitle(modelTitle[key].replace('\n',' ') + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

In [ ]:
method = 'mordm'

axes = robust_parallel_axes(robustData[method]['dps'][0], primaryColors[key]['mordm'])

axes.plot(robustData[method]['dps'][0],color=primaryColors['dps'][method], label=modelTitle['dps'].replace('\n',' '), linewidth=linewidth) 
axes.plot(robustData[method]['intertemporal'][0],color=primaryColors['intertemporal'][method], label=modelTitle['intertemporal'].replace('\n',' '), linewidth=linewidth)
axes.plot(robustData[method]['plannedadaptive'][0],color=primaryColors['plannedadaptive'][method], label=modelTitle['plannedadaptive'].replace('\n',' '), linewidth=linewidth)

axes = reorderDatalabels(kind='model', axes=axes)
axes.legend()
axes.fig.suptitle(methodTitle[method] + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

In [ ]:
method = 'multi'

axes = robust_parallel_axes(robustData[method]['dps'][0], primaryColors[key]['mordm'])

axes.plot(robustData[method]['dps'][0],color=primaryColors['dps'][method], label=modelTitle['dps'].replace('\n',' '), linewidth=linewidth) 
axes.plot(robustData[method]['intertemporal'][0],color=primaryColors['intertemporal'][method], label=modelTitle['intertemporal'].replace('\n',' '), linewidth=linewidth)
axes.plot(robustData[method]['plannedadaptive'][0],color=primaryColors['plannedadaptive'][method], label=modelTitle['plannedadaptive'].replace('\n',' '), linewidth=linewidth)

axes = reorderDatalabels(kind='model', axes=axes)
axes.legend()
axes.fig.suptitle(methodTitle[method] + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

In [ ]:
method = 'moro'

axes = robust_parallel_axes(robustData[method]['dps'][0], primaryColors[key]['mordm'])

axes.plot(robustData[method]['dps'][0],color=primaryColors['dps'][method], label=modelTitle['dps'].replace('\n',' '), linewidth=linewidth) 
axes.plot(robustData[method]['intertemporal'][0],color=primaryColors['intertemporal'][method], label=modelTitle['intertemporal'].replace('\n',' '), linewidth=linewidth)
axes.plot(robustData[method]['plannedadaptive'][0],color=primaryColors['plannedadaptive'][method], label=modelTitle['plannedadaptive'].replace('\n',' '), linewidth=linewidth)

axes = reorderDatalabels(kind='model', axes=axes)
axes.legend()
axes.fig.suptitle(methodTitle[method] + ' Comparison', fontsize=16, weight='bold', y = 1.05)
plt.show()

# Robust Value Ranges

In [ ]:
def showRobustRanges(model, robustData):
    leverVals = []
    for method in order('method'):
        data = pd.concat(robustData[method][model])
        for rob in data.columns: 
            robVals = data[rob]
            df = pd.DataFrame(robVals)
            df.columns = ['value']
            df['method'] = method
            df['robust_outcome'] = rob
            leverVals.append(df)

    df = pd.concat(leverVals)
    pal = {'mordm': primaryColors[model]['mordm'],'multi': primaryColors[model]['multi'],'moro': primaryColors[model]['moro']}
    for tp in ['violin','box']: 
        g = sns.factorplot(x="method", y="value", col="robust_outcome", palette=pal,
                           data=df, kind=tp,
                           size=4, aspect=.7).set_titles("{col_name}".replace('\n',' '))    
        g.set_axis_labels('', "Value").set_xticklabels(['MORDM','Multi-Scenario\nMORDM','MORO'])
        plt.suptitle('Robust Outcome Ranges for ' + modelTitle[model].replace('\n',' '), fontsize=16,weight='bold',y=1.05)
        
        save_fig(g, 'images/robust/', 'outcomeranges_' + tp + '_' + model)
        
        plt.show()

In [ ]:
showRobustRanges('dps', robustData)

In [ ]:
showRobustRanges('plannedadaptive', robustData)

In [ ]:
showRobustRanges('intertemporal', robustData)